# Scaling Domains

> This demo notebook showcases some key functions for scaling coordinates. 
> We often need to use semantic coordinate values that are actually useful for computation. 
> So for example, lat/lon coordinates are often in the spherical domain. But for many physical problems, we need them in meters. 
> Similarly with time coordinates: we often have `np.datetime64` coordinate values but we need them in seconds.

**Note**: There are some more meaningful rescaling that involve the mean and variance (`StandardScaler`) or specific min/max values (`MinMaxScaler`) but this is an arbitrary transformation that isn't specific to coordinate values.

In [1]:
import autoroot
import typing as tp
from dataclasses import dataclass
import numpy as np
import pandas as pd
import xarray as xr
import einops
import xarray_dataclasses as xrdataclass
from oceanbench._src.datasets.base import XRDABatcher

%load_ext autoreload
%autoreload 2


In [2]:
file = "/gpfswork/rech/cli/uvo53rl/projects/jejeqx/data/natl60/NATL60-CJM165_GULFSTREAM_ssh_y2013.1y.nc"
!ls $file

/gpfswork/rech/cli/uvo53rl/projects/jejeqx/data/natl60/NATL60-CJM165_GULFSTREAM_ssh_y2013.1y.nc


In [3]:
da = xr.open_dataset(file, decode_times=False)
da["time"] = pd.to_datetime(da.time)
da = da.sortby("time")

## Spatial Rescaling

#### Degrees to Meters

In [4]:
from oceanbench._src.geoprocessing.spatial import latlon_deg2m


In [9]:
da_scaled = latlon_deg2m(da, mean=False)
da_scaled

<xarray.Dataset>
Dimensions:  (time: 365, lat: 201, lon: 201)
Coordinates:
  * lon      (lon) float64 0.0 4.66e+03 9.318e+03 ... 8.708e+05 8.748e+05
  * lat      (lat) float64 0.0 5.56e+03 1.112e+04 ... 1.106e+06 1.112e+06
  * time     (time) datetime64[ns] 2012-10-01 2012-10-02 ... 2013-09-30
Data variables:
    ssh      (time, lat, lon) float64 ...

## Temporal Rescaling

#### DateTime 2 Seconds

In [10]:
from oceanbench._src.geoprocessing.temporal import time_rescale
import pandas as pd

In [11]:
t0 = "2012-12-15"
freq_dt = 1
freq_unit = "D"


da_scale = time_rescale(da_scaled, freq_dt=freq_dt, freq_unit=freq_unit, t0=t0)

da_scale

<xarray.Dataset>
Dimensions:  (time: 365, lat: 201, lon: 201)
Coordinates:
  * lon      (lon) float64 0.0 4.66e+03 9.318e+03 ... 8.708e+05 8.748e+05
  * lat      (lat) float64 0.0 5.56e+03 1.112e+04 ... 1.106e+06 1.112e+06
  * time     (time) float32 -75.0 -74.0 -73.0 -72.0 ... 286.0 287.0 288.0 289.0
Data variables:
    ssh      (time, lat, lon) float64 ...